<a href="https://colab.research.google.com/github/shirleyzhang2/Autobuilder-2.0/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIV1499 - Introduction to Data Science
## Project - Toronto Bike Share

Insert description of project here

In [1]:
#COLAB ONLY
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup Notebook

In [2]:
# Import 3rd party libraries
import os
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

# 1. Weather Data

The City of Toronto provided a large number of csv files containing historical weather in Toronto. The weather station is located at  43.63 latitudes and -79.4 longitude. Here are the names for some of the files.

In [3]:
# # JUPYTER
# weather_filenames = [filename for filename in os.listdir() if 'climate' in filename]
# print(weather_filenames[0:5])

In [4]:
# COLAB
# SET THE PATH HERE TO THE LOCATION OF YOUR CSV FILES
path = r'/content/drive/MyDrive/CIV1498_project/'
weather_filenames = [filename for filename in os.listdir(path) if 'climate' in filename]
print(weather_filenames[0:5])

['en_climate_hourly_ON_6158359_06-2018_P1H.csv', 'en_climate_hourly_ON_6158359_09-2018_P1H.csv', 'en_climate_hourly_ON_6158359_07-2017_P1H.csv', 'en_climate_hourly_ON_6158359_01-2018_P1H.csv', 'en_climate_hourly_ON_6158359_03-2018_P1H.csv']


# 2. Bike Share Trip Data

Bike Share Toronto has provided a dataset of historical ridership data in Toronto from 2017, 2018, 2019, and 2020. We will organize this dataset into a dictionary of dataframes. Note: Displaying column headers for the 2019 and 2020 data reveals that there are 2 spaces in `'Trip  Duration'`!

In [5]:
# # JUPYTER
# # Create a list of file names for bike share trips data
# trips_filenames = [filename for filename in os.listdir() if 'bike_share' in filename]

# # Create a dictionary where key:value pairs correspond to 
# # the file name and DataFrame respectively
# trips_data = {filename: pd.read_csv(filename) for filename in trips_filenames}

# # Now lets print out the column names for the first month of each year
# print('2017\n{}\n'.format(trips_data['bike_share_2017-1.csv'].columns.tolist()))
# print('2018\n{}\n'.format(trips_data['bike_share_2018-1.csv'].columns.tolist()))
# print('2019\n{}\n'.format(trips_data['bike_share_2019-1.csv'].columns.tolist()))
# print('2020\n{}\n'.format(trips_data['bike_share_2020-1.csv'].columns.tolist()))

In [6]:
# COLAB
# Create a list of file names for bike share trips data
trips_filenames = [filename for filename in os.listdir(path) if 'bike_share' in filename]

# Create a dictionary where key:value pairs correspond to 
# the file name and DataFrame respectively
trips_data = {filename: pd.read_csv(path+filename) for filename in trips_filenames}

# Now lets print out the column names for the first month of each year
print('2017\n{}\n'.format(trips_data['bike_share_2017-1.csv'].columns.tolist()))
print('2018\n{}\n'.format(trips_data['bike_share_2018-1.csv'].columns.tolist()))
print('2019\n{}\n'.format(trips_data['bike_share_2019-1.csv'].columns.tolist()))
print('2020\n{}\n'.format(trips_data['bike_share_2020-1.csv'].columns.tolist()))

2017
['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'user_type']

2018
['trip_id', 'trip_duration_seconds', 'from_station_id', 'trip_start_time', 'from_station_name', 'trip_stop_time', 'to_station_id', 'to_station_name', 'user_type']

2019
['Trip Id', 'Subscription Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Station Id', 'End Time', 'End Station Name', 'Bike Id', 'User Type']

2020
['Trip Id', 'Subscription Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Station Id', 'End Time', 'End Station Name', 'Bike Id', 'User Type']



# 3. Station Data

Station data from the Bike Share API endpoint will also be converted into a dataframe.

In [7]:
# # JUPYTER
# stations = pd.read_csv('bikeshare_stations.csv')
# stations.head()

In [8]:
# COLAB
stations = pd.read_csv(path+'bikeshare_stations.csv')
stations.head()

,Station Id,Station Name,lat,lon,capacity
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,35
1,7001,Lower Jarvis St / The Esplanade,43.647830,-79.370698,15
2,7002,St. George St / Bloor St W,43.667333,-79.399429,19
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,15
4,7004,University Ave / Elm St,43.656518,-79.389099,11


# 4. Data Cleaning & Wrangling
The raw data from the City of Toronto contains weather, bikeshare trips, and station information from 2017 to 2020. Some inconsistencies that can be observed across the files include station names, outliers, missing data, DateTime formats, and timezones. The first step is to clean and merge this data into a common DataFrame. Any issues with the datasets will be addressed and documented. The final output will be a DataFrame including bike share and weather data from 2017 to 2020 with each row corresponding to a trip. 

## 4a. Unify column headers

The column names for the 2017-2018 data are different from that of the 2019-2020 data. For example, trip identifiers are represented as `'trip_id'` in 2017-2018 and as `'Trip Id'` in 2019-2020. The order of the columns are also different. In addition, `'Subscription Id'` and `'Bike Id'` are not in the 2017-2018 datasets. Column headers from 2017-2018 will be modified to match the most recent data, and two empty columns will be created as placeholders for the subscription and bike identifiers.

In [9]:
import numpy as np

new_2017_headers = ['Trip Id', 'Start Time', 'End Time', 'Trip  Duration', 'Start Station Id', 'Start Station Name', 'End Station Id', 'End Station Name', 'User Type']
new_2018_headers = ['Trip Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Time', 'End Station Id', 'End Station Name', 'User Type']

for filename in trips_data:
    if '2017' in filename:
        # Modify 2017 headers
        trips_data[filename].columns = new_2017_headers
        # Add subscription and bike Id columns filled with NaN
        trips_data[filename]['Subscription Id'] = np.nan
        trips_data[filename]['Bike Id'] = np.nan 
        # Rearrange 2017 headers to match 2019-1020 data
        trips_data[filename] = trips_data[filename][['Trip Id', 'Subscription Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Station Id', 'End Time', 'End Station Name', 'Bike Id', 'User Type']]

    elif '2018' in filename:
        # Modify 2018 headers
        trips_data[filename].columns = new_2018_headers
        # Add subscription and bike Id columns filled with NaN
        trips_data[filename]['Subscription Id'] = np.nan
        trips_data[filename]['Bike Id'] = np.nan 
        # Rearrange 2018 headers to match 2019-1020 data 
        trips_data[filename] = trips_data[filename][['Trip Id', 'Subscription Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Station Id', 'End Time', 'End Station Name', 'Bike Id', 'User Type']]

# # View a sample DataFrame
trips_data['bike_share_2017-1.csv'].head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,712441,NaN,274,7006.0,2017-01-01 05:03 (UTC),Bay St / College St (East Side),7021.0,2017-01-01 05:08 (UTC),Bay St / Albert St,NaN,Member
1,712442,NaN,538,7046.0,2017-01-01 05:03 (UTC),Niagara St / Richmond St W,7147.0,2017-01-01 05:12 (UTC),King St W / Fraser Ave,NaN,Member
2,712443,NaN,992,7048.0,2017-01-01 05:05 (UTC),Front St / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 05:22 (UTC),Church St / Wood St,NaN,Member
3,712444,NaN,1005,7177.0,2017-01-01 05:09 (UTC),East Liberty St / Pirandello St,7202.0,2017-01-01 05:26 (UTC),Queen St W / York St (City Hall),NaN,Member
4,712445,NaN,645,7203.0,2017-01-01 05:14 (UTC),Bathurst St / Queens Quay W,7010.0,2017-01-01 05:25 (UTC),King St W / Spadina Ave,NaN,Member


## 4b. Unify Datetimes

It was also observed that the 2017 and 2018 datasets display `'Start Time'` and `'End Time'` in UTC and in the format of Y/m/d H:M, while the 2019 and 2020 datasets display them in Eastern Standard Time (EST) in the format of d/m/Y H:M:S. For consistency, all `'Start Time'` and `'End Time'` will be converted to datetime objects and localized to EST. Note that by setting `error='coerce'`, invalid parsing will be set as NaT instead of raising an exception.

In [10]:
from datetime import timedelta

for filename in trips_data:
   if '2017' in filename or '2018' in filename:
    # strip (UTC) or (EST) at end of string before performing datetime maniputations
    trips_data[filename]['Start Time'] = pd.to_datetime(trips_data[filename]['Start Time']
                                                         .astype('str').str.rstrip(' (UTC)'), format = '%Y/%m/%d %H:%M',\
                                                            errors='coerce').dt.tz_localize('UTC').dt.tz_convert('EST')
    trips_data[filename]['End Time'] = pd.to_datetime(trips_data[filename]['End Time']
                                                         .astype('str').str.rstrip(' (UTC)'), format = '%Y/%m/%d %H:%M',\
                                                            errors='coerce').dt.tz_localize('UTC').dt.tz_convert('EST')
   elif '2019' in filename or '2020' in filename:
    trips_data[filename]['Start Time'] = pd.to_datetime(trips_data[filename]['Start Time']
                                                         .astype('str').str.rstrip(' (EST)'), format = '%d/%m/%Y %H:%M:%S',\
                                                            errors='coerce').dt.tz_localize('EST')
    trips_data[filename]['End Time'] = pd.to_datetime(trips_data[filename]['End Time']
                                                         .astype('str').str.rstrip(' (EST)'), format = '%d/%m/%Y %H:%M:%S',\
                                                            errors='coerce').dt.tz_localize('EST')
# View DataFrame
trips_data['bike_share_2017-1.csv'].head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,712441,NaN,274,7006.0,2017-01-01 00:03:00-05:00,Bay St / College St (East Side),7021.0,2017-01-01 00:08:00-05:00,Bay St / Albert St,NaN,Member
1,712442,NaN,538,7046.0,2017-01-01 00:03:00-05:00,Niagara St / Richmond St W,7147.0,2017-01-01 00:12:00-05:00,King St W / Fraser Ave,NaN,Member
2,712443,NaN,992,7048.0,2017-01-01 00:05:00-05:00,Front St / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 00:22:00-05:00,Church St / Wood St,NaN,Member
3,712444,NaN,1005,7177.0,2017-01-01 00:09:00-05:00,East Liberty St / Pirandello St,7202.0,2017-01-01 00:26:00-05:00,Queen St W / York St (City Hall),NaN,Member
4,712445,NaN,645,7203.0,2017-01-01 00:14:00-05:00,Bathurst St / Queens Quay W,7010.0,2017-01-01 00:25:00-05:00,King St W / Spadina Ave,NaN,Member


## 4c. Combine trip dataframes

Now that the column headers and datetimes are unified, we can concatenate the bike share trip dataframes to create a single dataframe representing bike share trip data from 2017 to 2020.

In [11]:
all_trips = pd.concat(trips_data.values(), ignore_index=True).sort_values(by=['Trip Id']).reset_index(drop=True)
all_trips.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,712382,NaN,223,7051.0,2016-12-31 20:00:00-05:00,Wellesley St E / Yonge St Green P,7089.0,2016-12-31 20:03:00-05:00,Church St / Wood St,NaN,Member
1,712383,NaN,279,7143.0,2016-12-31 20:00:00-05:00,Kendal Ave / Bernard Ave,7154.0,2016-12-31 20:05:00-05:00,Bathurst Subway Station,NaN,Member
2,712384,NaN,1394,7113.0,2016-12-31 20:05:00-05:00,Parliament St / Aberdeen Ave,7199.0,2016-12-31 20:29:00-05:00,College St W / Markham St,NaN,Member
3,712385,NaN,826,7077.0,2016-12-31 20:07:00-05:00,College Park South,7010.0,2016-12-31 20:21:00-05:00,King St W / Spadina Ave,NaN,Member
4,712386,NaN,279,7079.0,2016-12-31 20:08:00-05:00,McGill St / Church St,7047.0,2016-12-31 20:12:00-05:00,University Ave / Gerrard St W,NaN,Member


After organizing the trips in the `'all_trips'` dataframe by ascending trip Ids, we noticed that some trips actually occured at the end of 2016. Further investigation reveals that 'bike_share_2017-12.csv' contains some datapoints that are in 2017 according to UTC but in 2016 according to EST. Since we are primarily interested in data from 2017 to 2020, we will remove the 2016 data.

In [12]:
all_trips = all_trips[~(all_trips['Start Time'] < '2017-01-01')]

# View DataFrame
all_trips.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
58,712441,NaN,274,7006.0,2017-01-01 00:03:00-05:00,Bay St / College St (East Side),7021.0,2017-01-01 00:08:00-05:00,Bay St / Albert St,NaN,Member
59,712442,NaN,538,7046.0,2017-01-01 00:03:00-05:00,Niagara St / Richmond St W,7147.0,2017-01-01 00:12:00-05:00,King St W / Fraser Ave,NaN,Member
60,712443,NaN,992,7048.0,2017-01-01 00:05:00-05:00,Front St / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 00:22:00-05:00,Church St / Wood St,NaN,Member
61,712444,NaN,1005,7177.0,2017-01-01 00:09:00-05:00,East Liberty St / Pirandello St,7202.0,2017-01-01 00:26:00-05:00,Queen St W / York St (City Hall),NaN,Member
62,712445,NaN,645,7203.0,2017-01-01 00:14:00-05:00,Bathurst St / Queens Quay W,7010.0,2017-01-01 00:25:00-05:00,King St W / Spadina Ave,NaN,Member


## 4d. Clean up station data

We have noticed that some stations in the trip dataframe are missing station IDs. To further combine the trip data with station data, we need to resolve potential issues with duplicate or inconsistent station names. We will first create a new dataframe containing unique station ID and station name combinations.

In [13]:
# Create dataframe for start station Ids and start station names
start_station = all_trips[['Start Station Id', 'Start Station Name']].rename(columns={'Start Station Id':'Station Id', 'Start Station Name':'Station Name'})

# Create dataframe for end station Ids and end station names
end_station = all_trips[['End Station Id', 'End Station Name']].rename(columns={'End Station Id':'Station Id', 'End Station Name':'Station Name'})

# Concatenate the dataframes and drop missing values as well as duplicates
all_stations = pd.concat([start_station, end_station]).dropna(how='all').drop_duplicates().reset_index(drop=True)

# View DataFrame
all_stations.head()

,Station Id,Station Name
0,7006.0,Bay St / College St (East Side)
1,7046.0,Niagara St / Richmond St W
2,7048.0,Front St / Yonge St (Hockey Hall of Fame)
3,7177.0,East Liberty St / Pirandello St
4,7203.0,Bathurst St / Queens Quay W


Let's check if there is any mismatch between station names and station IDs.

In [14]:
all_stations.isna().sum()

Station Id      271
Station Name      2
dtype: int64

Looks like quite a few stations in the trip dataframe don't have corresponding station IDs. We will make a separate dataframe for these stations.

In [15]:
missing_Id = all_stations[all_stations['Station Id'].isnull()].reset_index(drop=True)
missing_Id.head()

,Station Id,Station Name
0,NaN,Princess St / Adelaide St E
1,NaN,Fort York Blvd / Capreol Crt
2,NaN,Elizabeth St / Edward St (Bus Terminal)
3,NaN,Bay St / St. Joseph St
4,NaN,Bay St / College St (East Side)


Using the Fussywussy library, let's try to find the missing IDs by matching the station names with the API station data from Part 3.

In [16]:
# Download fussywussy
import sys
!{sys.executable} -m pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [17]:
for Id, name in missing_Id.iterrows():
    for station_Id, station_name in stations.iterrows():
        ratio = fuzz.ratio(name['Station Name'], station_name['Station Name'])
        # Fill in station Ids for station names that are similar enough to the API data
        if ratio > 85:
            missing_Id.at[Id, 'Station Id'] = station_name['Station Id']

# Remove stations that could not be matched        
missing_Id = missing_Id.dropna()

# View DataFrame
missing_Id.head()

,Station Id,Station Name
0,7060.0,Princess St / Adelaide St E
1,7000.0,Fort York Blvd / Capreol Crt
2,7012.0,Elizabeth St / Edward St (Bus Terminal)
3,7026.0,Bay St / St. Joseph St
4,7006.0,Bay St / College St (East Side)


Now, we will add the new-found station IDs to `'all_stations'`, and merge this dataframe with the station data.

In [18]:
# Create a dataframe for datapoints that had existing station Ids
has_Id = all_stations[all_stations['Station Id'].notnull()]

# Combine this dataframe with the one above (stations with newly added Ids) and merge with API data
all_stations = pd.concat([has_Id, missing_Id]).merge(stations.drop(columns=['Station Name']), how='inner', on='Station Id').drop_duplicates()

# View DataFrame
all_stations.head()

,Station Id,Station Name,lat,lon,capacity
0,7006.0,Bay St / College St (East Side),43.660439,-79.385525,11
2,7046.0,Niagara St / Richmond St W,43.645340,-79.409597,26
4,7048.0,Front St / Yonge St (Hockey Hall of Fame),43.646144,-79.377962,47
5,7048.0,Front St W / Yonge St (Hockey Hall of Fame),43.646144,-79.377962,47
7,7177.0,East Liberty St / Pirandello St,43.638611,-79.414722,19


Then, we will merge the cleaned station data with the trip data for both start and end stations. Note that duplicate columns are dropped.

In [19]:
all_trips = all_trips.merge(all_stations, how='inner', left_on='Start Station Name', right_on='Station Name')\
                    .rename(columns={"lat": "Start lat", "lon": "Start lon", "capacity": "Start capacity"}).drop(columns=['Station Name', 'Station Id'])

all_trips = all_trips.merge(all_stations, how='inner', left_on='End Station Name', right_on='Station Name')\
                     .rename(columns={"lat": "End lat", "lon": "End lon", "capacity": "End capacity"}).drop(columns=['Station Name', 'Station Id'])

all_trips = all_trips.sort_values(by=['Trip Id']).reset_index(drop=True).drop_duplicates(subset=['Trip Id'])

# View DataFrame
all_trips.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,Start lat,Start lon,Start capacity,End lat,End lon,End capacity
0,712441,NaN,274,7006.0,2017-01-01 00:03:00-05:00,Bay St / College St (East Side),7021.0,2017-01-01 00:08:00-05:00,Bay St / Albert St,NaN,Member,43.660439,-79.385525,11,43.653264,-79.382458,35
1,712442,NaN,538,7046.0,2017-01-01 00:03:00-05:00,Niagara St / Richmond St W,7147.0,2017-01-01 00:12:00-05:00,King St W / Fraser Ave,NaN,Member,43.645340,-79.409597,26,43.639444,-79.425278,15
2,712443,NaN,992,7048.0,2017-01-01 00:05:00-05:00,Front St / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 00:22:00-05:00,Church St / Wood St,NaN,Member,43.646144,-79.377962,47,43.662712,-79.379903,19
3,712444,NaN,1005,7177.0,2017-01-01 00:09:00-05:00,East Liberty St / Pirandello St,7202.0,2017-01-01 00:26:00-05:00,Queen St W / York St (City Hall),NaN,Member,43.638611,-79.414722,19,43.651667,-79.384167,31
4,712445,NaN,645,7203.0,2017-01-01 00:14:00-05:00,Bathurst St / Queens Quay W,7010.0,2017-01-01 00:25:00-05:00,King St W / Spadina Ave,NaN,Member,43.635492,-79.398253,35,43.645323,-79.395003,19


## 4e. Remove short trips and outliers

The next step is to remove trips with unreasonable durations, which are indications of false starts or incomplete trips. We will remove trips that are shorter than 60 seconds or are +/- 1.5 * IQR (interquartile range) away from the median duration.

In [20]:
# Remove short trips
all_trips = all_trips[all_trips['Trip  Duration'] >= 60]

# Calculate IQR
Q1 = all_trips['Trip  Duration'].quantile(0.25)
Q3 = all_trips['Trip  Duration'].quantile(0.75)
IQR = Q3-Q1
whisk_low = Q1-1.5*IQR
whisk_high = Q3+1.5*IQR

# Remove outliers
all_trips = all_trips.loc[(all_trips['Trip  Duration'] >= whisk_low) & (all_trips['Trip  Duration'] <= whisk_high)]

# View DataFrame
all_trips.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,Start lat,Start lon,Start capacity,End lat,End lon,End capacity
0,712441,NaN,274,7006.0,2017-01-01 00:03:00-05:00,Bay St / College St (East Side),7021.0,2017-01-01 00:08:00-05:00,Bay St / Albert St,NaN,Member,43.660439,-79.385525,11,43.653264,-79.382458,35
1,712442,NaN,538,7046.0,2017-01-01 00:03:00-05:00,Niagara St / Richmond St W,7147.0,2017-01-01 00:12:00-05:00,King St W / Fraser Ave,NaN,Member,43.645340,-79.409597,26,43.639444,-79.425278,15
2,712443,NaN,992,7048.0,2017-01-01 00:05:00-05:00,Front St / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 00:22:00-05:00,Church St / Wood St,NaN,Member,43.646144,-79.377962,47,43.662712,-79.379903,19
3,712444,NaN,1005,7177.0,2017-01-01 00:09:00-05:00,East Liberty St / Pirandello St,7202.0,2017-01-01 00:26:00-05:00,Queen St W / York St (City Hall),NaN,Member,43.638611,-79.414722,19,43.651667,-79.384167,31
4,712445,NaN,645,7203.0,2017-01-01 00:14:00-05:00,Bathurst St / Queens Quay W,7010.0,2017-01-01 00:25:00-05:00,King St W / Spadina Ave,NaN,Member,43.635492,-79.398253,35,43.645323,-79.395003,19
